In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data=pd.read_csv('data/train.csv')
test_data=pd.read_csv('data/test.csv')
sub_data=pd.read_csv('data/Sample_Submission.csv')

In [3]:
train_data.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Marital_Status                550068 non-null int64
Product_Category_1            550068 non-null int64
Product_Category_2            376430 non-null float64
Product_Category_3            166821 non-null float64
Purchase                      550068 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [5]:
train_data['is_male']=train_data['Gender'].apply(lambda x: 1 if str(x)=='M' else 0)
test_data['is_male']=test_data['Gender'].apply(lambda x: 1 if str(x)=='M' else 0)

In [6]:
train_data['Product_Category_2']=train_data['Product_Category_2'].fillna(0)
test_data['Product_Category_2']=test_data['Product_Category_2'].fillna(0)

In [7]:
train_data['Product_Category_3']=train_data['Product_Category_3'].fillna(0)
test_data['Product_Category_3']=test_data['Product_Category_3'].fillna(0)

In [8]:
train_data['city_A']=train_data['City_Category'].apply(lambda x: 1 if str(x)=='A' else 0)
test_data['city_A']=test_data['City_Category'].apply(lambda x: 1 if str(x)=='A' else 0)
train_data['city_B']=train_data['City_Category'].apply(lambda x: 1 if str(x)=='B' else 0)
test_data['city_B']=test_data['City_Category'].apply(lambda x: 1 if str(x)=='B' else 0)
train_data['city_C']=train_data['City_Category'].apply(lambda x: 1 if str(x)=='C' else 0)
test_data['city_C']=test_data['City_Category'].apply(lambda x: 1 if str(x)=='C' else 0)

In [9]:
def age_lower(val):
    if str(val)=='0-17':
        return 0
    elif str(val)=='18-25':
        return 18
    elif str(val)=='26-35':
        return 26
    elif str(val)=='36-45':
        return 36
    elif str(val)=='46-50':
        return 46
    elif str(val)=='51-55':
        return 51
    else:
        return 56
    
def age_upper(val):
    if str(val)=='0-17':
        return 17
    elif str(val)=='18-25':
        return 25
    elif str(val)=='26-35':
        return 35
    elif str(val)=='36-45':
        return 45
    elif str(val)=='46-50':
        return 50
    elif str(val)=='51-55':
        return 55
    else:
        return 85

In [10]:
train_data['Age_Lower']=train_data['Age'].apply(lambda x: age_lower(x))
test_data['Age_Lower']=test_data['Age'].apply(lambda x: age_lower(x))
train_data['Age_Upper']=train_data['Age'].apply(lambda x: age_upper(x))
test_data['Age_Upper']=test_data['Age'].apply(lambda x: age_upper(x))

In [11]:
def stay(val):
    if str(val)=='4+':
        return 7
    return int(val)     

In [12]:
train_data['stay_years']=train_data['Stay_In_Current_City_Years'].apply(lambda x: stay(x))
test_data['stay_years']=test_data['Stay_In_Current_City_Years'].apply(lambda x: stay(x))

In [13]:
train_data.drop(['Stay_In_Current_City_Years','City_Category','Gender','Age'],axis = 1, inplace=True)
test_data.drop(['Stay_In_Current_City_Years','City_Category','Gender','Age'],axis = 1, inplace=True)

In [14]:
sub_data=train_data[['User_ID','Product_ID']]

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
col='Product_ID'
data=test_data[col].append(train_data[col])
le.fit(data.values)
train_data[col]=le.transform(train_data[col])
test_data[col]=le.transform(test_data[col])

In [16]:
train_data.drop(['User_ID'],axis = 1, inplace=True)
test_data.drop(['User_ID'],axis = 1, inplace=True)

In [17]:
train_purchase=train_data['Purchase']

In [18]:
train_data.drop(['Purchase'],axis = 1, inplace=True)

In [32]:
from sklearn import preprocessing
'''
scaler = preprocessing.StandardScaler().fit(train_data)
train_data=scaler.transform(train_data)
min_max_scaler = preprocessing.MinMaxScaler()
train_data = min_max_scaler.fit_transform(train_data)
'''
normalizer = preprocessing.Normalizer().fit(train_data)
train_data=normalizer.transform(train_data)

In [33]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y =train_test_split(train_data, train_purchase, test_size=0.2, random_state=12)

In [34]:
from sklearn import linear_model
clf = linear_model.BayesianRidge()

In [35]:
clf.fit(train_x,train_y)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [36]:
pred=clf.predict(test_x)

In [37]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, pred)

21143002.940749604

In [81]:
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf_g = ensemble.GradientBoostingRegressor(**params)

In [ ]:
clf_g.fit(train_x,train_y)

In [67]:
pred_g=clf_g.predict(test_x)

In [68]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, pred_g)

8531895.6937988084

21227303.365345184

In [44]:
train_data['stay_years'].value_counts()

1    193821
2    101838
3     95285
7     84726
0     74398
Name: stay_years, dtype: int64

In [55]:
test_data.head(2)

,Product_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,is_male,city_A,city_B,city_C,Age_Lower,Age_Upper,stay_years
0,1216,7,1,1,11.0,0.0,1,0,1,0,46,50,2
1,1063,17,0,3,5.0,0.0,1,0,0,1,26,35,0


In [58]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233599 entries, 0 to 233598
Data columns (total 13 columns):
Product_ID            233599 non-null int64
Occupation            233599 non-null int64
Marital_Status        233599 non-null int64
Product_Category_1    233599 non-null int64
Product_Category_2    233599 non-null float64
Product_Category_3    233599 non-null float64
is_male               233599 non-null int64
city_A                233599 non-null int64
city_B                233599 non-null int64
city_C                233599 non-null int64
Age_Lower             233599 non-null int64
Age_Upper             233599 non-null int64
stay_years            233599 non-null int64
dtypes: float64(2), int64(11)
memory usage: 23.2 MB


In [53]:
train_data.groupby('Product_ID').count()

,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,is_male,city_A,city_B,city_C,Age_Lower,Age_Upper,stay_years
Product_ID,,,,,,,,,,,,,
0,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152
1,376,376,376,376,376,376,376,376,376,376,376,376,376
2,244,244,244,244,244,244,244,244,244,244,244,244,244
3,92,92,92,92,92,92,92,92,92,92,92,92,92
4,149,149,149,149,149,149,149,149,149,149,149,149,149
5,512,512,512,512,512,512,512,512,512,512,512,512,512
6,241,241,241,241,241,241,241,241,241,241,241,241,241
7,36,36,36,36,36,36,36,36,36,36,36,36,36
8,55,55,55,55,55,55,55,55,55,55,55,55,55


In [5]:
sub_data.head()

,User_ID,Product_ID,Purchase
